In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


This script is used to aggregate samples from different chunks of mESC data when SCRaPL is fitted with inflation. This is used as part of DIC estimation.

In [ ]:
from IPython import display
import pandas as pd
import numpy as np
import scipy
import scipy.stats

from matplotlib import pyplot as plt
from matplotlib import colors
from matplotlib.ticker import PercentFormatter
from tensorflow import keras

from tensorflow.keras import layers
import tensorflow_probability as tfp
import tensorflow.compat.v2 as tf
tf.enable_v2_behavior()

import pickle
from timeit import default_timer as timer

Folder = '/content/drive/MyDrive/'

In [ ]:
#For each one of the chunks store parameters of interest in a list.
cor_nuts_col = []
s_met_nuts_col = []
s_exp_nuts_col = []
m_met_nuts_col = []
m_exp_nuts_col = []
m_exp_nuts_col = []
inf_nuts_col = []
llk_nuts_col = []
x_exp_mn_nuts_col = []
x_met_mn_nuts_col = []

for prt in range(474):
    with open(Folder+'SCRaPL/Real/Results_DIC/nuts_cor_gastr_'+str(prt+1)+'.pickle', 'rb') as handle:
        cor_nuts_col.append(pickle.load(handle))
    with open(Folder+'SCRaPL/Real/Results_DIC/nuts_s_met_gastr_'+str(prt+1)+'.pickle', 'rb') as handle:
        s_met_nuts_col.append(pickle.load(handle))
    with open(Folder+'SCRaPL/Real/Results_DIC/nuts_s_exp_gastr_'+str(prt+1)+'.pickle', 'rb') as handle:
        s_exp_nuts_col.append(pickle.load(handle))
    with open(Folder+'SCRaPL/Real/Results_DIC/nuts_m_met_gastr_'+str(prt+1)+'.pickle', 'rb') as handle:
        m_met_nuts_col.append(pickle.load(handle))
    with open(Folder+'SCRaPL/Real/Results_DIC/nuts_m_exp_gastr_'+str(prt+1)+'.pickle', 'rb') as handle:
        m_exp_nuts_col.append(pickle.load(handle))
    with open(Folder+'SCRaPL/Real/Results_DIC/nuts_inf_gastr_'+str(prt+1)+'.pickle', 'rb') as handle:
        inf_nuts_col.append(pickle.load(handle))
    with open(Folder+'SCRaPL/Real/Results_DIC/log_prob_gastr_'+str(prt+1)+'.pickle', 'rb') as handle:
        llk_nuts_col.append(pickle.load(handle))
    with open(Folder+'SCRaPL/Real/Results_DIC/avg_exp_inf_gastr_'+str(prt+1)+'.pickle', 'rb') as handle:
        x_exp_mn_nuts_col.append(pickle.load(handle))
    with open(Folder+'SCRaPL/Real/Results_DIC/avg_met_inf_gastr_'+str(prt+1)+'.pickle', 'rb') as handle:
        x_met_mn_nuts_col.append(pickle.load(handle))

    print(prt/474)

0.0
0.002109704641350211
0.004219409282700422
0.006329113924050633
0.008438818565400843
0.010548523206751054
0.012658227848101266
0.014767932489451477
0.016877637130801686
0.0189873417721519
0.02109704641350211
0.023206751054852322
0.02531645569620253
0.027426160337552744
0.029535864978902954
0.03164556962025317
0.03375527426160337
0.035864978902953586
0.0379746835443038
0.04008438818565401
0.04219409282700422
0.04430379746835443
0.046413502109704644
0.04852320675105485
0.05063291139240506
0.052742616033755275
0.05485232067510549
0.056962025316455694
0.05907172995780591
0.06118143459915612
0.06329113924050633
0.06540084388185655
0.06751054852320675
0.06962025316455696
0.07172995780590717
0.07383966244725738
0.0759493670886076
0.07805907172995781
0.08016877637130802
0.08227848101265822
0.08438818565400844
0.08649789029535865
0.08860759493670886
0.09071729957805907
0.09282700421940929
0.0949367088607595
0.0970464135021097
0.09915611814345991
0.10126582278481013
0.10337552742616034
0.1054

In [ ]:
#Concat all lists to aggregate all samples.
cor_nuts = tf.concat(cor_nuts_col,axis=-1)
s_met_nuts = tf.concat(s_met_nuts_col,axis=-1)
s_exp_nuts = tf.concat(s_exp_nuts_col,axis=-1)
m_met_nuts = tf.concat(m_met_nuts_col,axis=-1)
m_exp_nuts = tf.concat(m_exp_nuts_col,axis=-1)
inf_nuts = tf.concat(inf_nuts_col,axis=-1)
llk_nuts = tf.concat(llk_nuts_col,axis=-1)
x_exp_mn_nuts = tf.concat(x_exp_mn_nuts_col,axis=-2)
x_met_mn_nuts = tf.concat(x_met_mn_nuts_col,axis=-2)

In [ ]:
#Save parameters of interest.
with open(Folder+'SCRaPL/Real/Results_DIC_col/nuts_cor_gastr_dic_n.pickle', 'wb') as handle:
    pickle.dump(cor_nuts, handle)
with open(Folder+'SCRaPL/Real/Results_DIC_col/nuts_m_met_gastr_dic_n.pickle', 'wb') as handle:
    pickle.dump(m_met_nuts, handle)
with open(Folder+'SCRaPL/Real/Results_DIC_col/nuts_m_exp_gastr_dic_n.pickle', 'wb') as handle:
    pickle.dump(m_exp_nuts, handle)
with open(Folder+'SCRaPL/Real/Results_DIC_col/nuts_s_met_gastr_dic_n.pickle', 'wb') as handle:
    pickle.dump(s_met_nuts, handle)
with open(Folder+'SCRaPL/Real/Results_DIC_col/nuts_s_exp_gastr_dic_n.pickle', 'wb') as handle:
    pickle.dump(s_exp_nuts, handle)
with open(Folder+'SCRaPL/Real/Results_DIC_col/nuts_inf_gastr_dic_n.pickle', 'wb') as handle:
    pickle.dump(inf_nuts, handle)
with open(Folder+'SCRaPL/Real/Results_DIC_col/nuts_llk_gastr_dic_n.pickle', 'wb') as handle:
    pickle.dump(llk_nuts, handle)
with open(Folder+'SCRaPL/Real/Results_DIC_col/nuts_x_met_mn_avg_dic_n.pickle', 'wb') as handle:
    pickle.dump(x_met_mn_nuts, handle)
with open(Folder+'SCRaPL/Real/Results_DIC_col/nuts_x_exp_mn_avg_dic_n.pickle', 'wb') as handle:
    pickle.dump(x_exp_mn_nuts, handle)
